In [82]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l1_l2
import keras
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [69]:
import zipfile
with zipfile.ZipFile('valoracion_aerolineas.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

df_train = pd.read_csv('/content/dataset/train.csv')
df_test = pd.read_csv('/content/dataset/test.csv')

In [70]:
df_train = df_train.dropna()
df_train = df_train.drop(columns=['Unnamed: 0', 'id'])
df_test = df_test.dropna()
df_test = df_test.drop(columns=['Unnamed: 0', 'id'])
df_train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,3,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
103901,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,3,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,5,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [71]:
df_train_dummie = df_train.copy()

# Aplicar Label Encoding a las columnas categóricas
label_encoder = LabelEncoder()
for column in df_train_dummie.select_dtypes(include=['object']).columns:
    df_train_dummie[column] = label_encoder.fit_transform(df_train_dummie[column])

df_test_dummie = df_test.copy()

# Aplicar Label Encoding a las columnas categóricas
label_encoder = LabelEncoder()
for column in df_test_dummie.select_dtypes(include=['object']).columns:
    df_test_dummie[column] = label_encoder.fit_transform(df_test_dummie[column])

In [72]:
df_train_selected = df_train_dummie[['Type of Travel','Class','Online boarding','satisfaction']]
df_test_selected = df_test_dummie[['Type of Travel','Class','Online boarding','satisfaction']]

In [74]:
# Age cluster
df_train_new_features = pd.DataFrame()
age_bins = [7,18,26,33,40,47,54,63,86]
df_train_new_features['Age Cluster'] = pd.cut(df_train_dummie['Age'], bins=age_bins, labels=False, right=False)

# Weight Comfort Seats
df_train_new_features['Weight Comfort Seats'] = (df_train_dummie['Seat comfort']/5 + df_train_dummie['Class'] + df_train_dummie['Type of Travel'])

# Media de todos los servicios que tienen valor del 0-5
df_train_new_features['Mean Satisfaction Services'] = df_train_dummie[['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink',
                                  'Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness']].mean(axis=1)

# Suma del Servicio en Vuelo
df_train_new_features['Sum Inflight Services'] = df_train_dummie['Inflight wifi service'] + df_train_dummie['Inflight service'] + df_train_dummie['Inflight entertainment'] + df_train_dummie['Online boarding']

# Peso sobre el espacio de los pies según la clase
df_train_new_features['Space Seat and Class'] = (df_train_dummie['Class'] * df_train_dummie['Leg room service']) / 5

# suma de servicios básicos
df_train_new_features['Weight Basic Services'] = df_train_dummie['Class'] + (df_train_dummie['Food and drink'] + df_train_dummie['Cleanliness'])/10


In [75]:
# Age clister
df_test_new_features = pd.DataFrame()
age_bins = [7,18,26,33,40,47,54,63,86]
df_test_new_features['Age Cluster'] = pd.cut(df_test['Age'], bins=age_bins, labels=False, right=False)

# Weight Comfort Seats
df_test_new_features['Weight Comfort Seats'] = (df_test_dummie['Seat comfort']/5 + df_test_dummie['Class'] + df_test_dummie['Type of Travel'])

# Media de todos los servicios que tienen valor del 0-5
df_test_new_features['Mean Satisfaction Services'] = df_test_dummie[['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink',
                                  'Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness']].mean(axis=1)

# Suma del Servicio en Vuelo
df_test_new_features['Sum Inflight Services'] = df_test_dummie['Inflight wifi service'] + df_test_dummie['Inflight service'] + df_test_dummie['Inflight entertainment'] + df_test_dummie['Online boarding']

# Peso sobre el espacio de los pies según la clase
df_test_new_features['Space Seat and Class'] = (df_test_dummie['Class'] * df_test_dummie['Leg room service']) / 5

# suma de servicios básicos
df_test_new_features['Weight Basic Services'] = df_test_dummie['Class'] + (df_test_dummie['Food and drink'] + df_test_dummie['Cleanliness'])/10

In [76]:
df_train_combined = pd.concat([df_train_selected, df_train_new_features], axis=1)

# Eliminar las columnas duplicadas si es necesario
df_train_combined = df_train_combined.loc[:, ~df_train_combined.columns.duplicated()]

df_test_combined = pd.concat([df_test_selected, df_test_new_features], axis=1)
df_test_combined = df_test_combined.loc[:, ~df_test_combined.columns.duplicated()]

# Callbacks

In [77]:
# Definir un callback personalizado para medir el tiempo de entrenamiento
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []
        self.train_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_start_time)

    def on_train_end(self, logs=None):
        self.total_train_time = time.time() - self.train_start_time
        
time_callback = TimeHistory()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

# Arquitectura 1: Valores sin análisis previo
- Inputs: Todo el data set (dummie)
- Epochs = 100
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate=0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32



In [96]:
X_train = df_train_dummie.drop('satisfaction', axis=1)
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie.drop('satisfaction', axis=1)
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Capas
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Hiperparámetros
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

#Recalls
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m1.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

# Modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 978us/step - loss: 0.1994 - accuracy: 0.9190 - val_loss: 0.1414 - val_accuracy: 0.9401 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 895us/step - loss: 0.1358 - accuracy: 0.9442 - val_loss: 0.1196 - val_accuracy: 0.9499 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1173 - accuracy: 0.9513 - val_loss: 0.1120 - val_accuracy: 0.9547 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 9s 3ms/step - loss: 0.1076 - accuracy: 0.9548 - val_loss: 0.1060 - val_accuracy: 0.9543 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 9s 4ms/step - loss: 0.1022 - accuracy: 0.9568 - val_loss: 0.1014 - val_accuracy: 0.9590 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.0971 - accuracy: 0.9588 - val_loss: 0.0975 - val_accuracy: 0.9586 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] 

In [98]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m1.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 129.60 segundos
Pérdida: 0.0660059005022049
Accuracy Modelo: 97.14509844779968
Recall: 0.9456225252969643
Specificity: 0.5707334
F1-Score: 0.9561387900355872
Precisión: 0.9668915879442195


# Arquitectura 2: Correlativa Individual
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 4ta capa = 1 sigmoid
- learning_rate=0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [106]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m2.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 944us/step - loss: 0.2931 - accuracy: 0.8818 - val_loss: 0.2595 - val_accuracy: 0.8973 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 888us/step - loss: 0.2601 - accuracy: 0.8950 - val_loss: 0.2463 - val_accuracy: 0.9022 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2514 - accuracy: 0.8979 - val_loss: 0.2415 - val_accuracy: 0.9021 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 899us/step - loss: 0.2456 - accuracy: 0.9007 - val_loss: 0.2400 - val_accuracy: 0.9025 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 915us/step - loss: 0.2404 - accuracy: 0.9016 - val_loss: 0.2357 - val_accuracy: 0.9044 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 2s 904us/step - loss: 0.2372 - accuracy: 0.9026 - val_loss: 0.2315 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [107]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m2.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 116.46 segundos
Pérdida: 0.2061804085969925
Accuracy Modelo: 91.41477942466736
Recall: 0.8807743070831501
Specificity: 0.57548374
F1-Score: 0.8954689806324642
Precisión: 0.9106622998544396


# Arquitectura 3: Correlaciones cercanas > 0.3 (+1 capa)
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 3ra capa = 16 relu
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [108]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m3.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 981us/step - loss: 0.2894 - accuracy: 0.8826 - val_loss: 0.2538 - val_accuracy: 0.8986 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 911us/step - loss: 0.2577 - accuracy: 0.8966 - val_loss: 0.2425 - val_accuracy: 0.9027 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 2s 932us/step - loss: 0.2478 - accuracy: 0.8998 - val_loss: 0.2486 - val_accuracy: 0.8993 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 946us/step - loss: 0.2407 - accuracy: 0.9020 - val_loss: 0.2305 - val_accuracy: 0.9071 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2354 - accuracy: 0.9039 - val_loss: 0.2290 - val_accuracy: 0.9065 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 3s 994us/step - loss: 0.2323 - accuracy: 0.9049 - val_loss: 0.2243 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [109]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m3.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 103.97 segundos
Pérdida: 0.20288875699043274
Accuracy Modelo: 91.50407314300537
Recall: 0.8815662120545534
Specificity: 0.5758699
F1-Score: 0.8966751689264779
Precisión: 0.9123110544527409


# Arquitectura 4: Correlaciones > 0.3 (Other activations)
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 tanh
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [110]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m4.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 885us/step - loss: 0.2997 - accuracy: 0.8778 - val_loss: 0.2665 - val_accuracy: 0.8943 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 890us/step - loss: 0.2654 - accuracy: 0.8923 - val_loss: 0.2490 - val_accuracy: 0.9006 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2541 - accuracy: 0.8969 - val_loss: 0.2430 - val_accuracy: 0.9012 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2462 - accuracy: 0.9006 - val_loss: 0.2363 - val_accuracy: 0.9046 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 883us/step - loss: 0.2411 - accuracy: 0.9016 - val_loss: 0.2343 - val_accuracy: 0.9038 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 2s 860us/step - loss: 0.2376 - accuracy: 0.9028 - val_loss: 0.2301 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 7/50
2590/2590 [============================

In [111]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m4.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 165.34 segundos
Pérdida: 0.2119370996952057
Accuracy Modelo: 91.231369972229
Recall: 0.8745270567531896
Specificity: 0.5805816
F1-Score: 0.894398200224972
Precisión: 0.9151933701657459


# Arquitectura 5: Correlaciones > 0.3 (Más nodos, más capas, nuevas funciones activación)

In [112]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m5.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 945us/step - loss: 0.2755 - accuracy: 0.8882 - val_loss: 0.2439 - val_accuracy: 0.9001 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 909us/step - loss: 0.2461 - accuracy: 0.8996 - val_loss: 0.2320 - val_accuracy: 0.9072 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2365 - accuracy: 0.9021 - val_loss: 0.2259 - val_accuracy: 0.9047 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 953us/step - loss: 0.2311 - accuracy: 0.9042 - val_loss: 0.2223 - val_accuracy: 0.9100 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 952us/step - loss: 0.2266 - accuracy: 0.9060 - val_loss: 0.2205 - val_accuracy: 0.9107 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 3s 974us/step - loss: 0.2241 - accuracy: 0.9074 - val_loss: 0.2194 - val_accuracy: 0.9102 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [113]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m5.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 146.32 segundos
Pérdida: 0.19842857122421265
Accuracy Modelo: 91.65852069854736
Recall: 0.8780466344038715
Specificity: 0.5789982
F1-Score: 0.8963442019222132
Precisión: 0.9154206036143473


# Arquitectura 5: Mayores niveles

In [16]:
X = df_train_dummie[['Online boarding','Inflight wifi service','Class','Type of Travel','Inflight entertainment','Seat comfort','Leg room service','Customer Type','Ease of Online booking']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo5.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 7s 3ms/step - loss: 0.4468 - accuracy: 0.8810 - val_loss: 0.2991 - val_accuracy: 0.9167
Epoch 2/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3252 - accuracy: 0.9028 - val_loss: 0.2694 - val_accuracy: 0.9214
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.3031 - accuracy: 0.9080 - val_loss: 0.2549 - val_accuracy: 0.9268
Epoch 4/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2893 - accuracy: 0.9113 - val_loss: 0.2418 - val_accuracy: 0.9324
Epoch 5/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2803 - accuracy: 0.9141 - val_loss: 0.2338 - val_accuracy: 0.9342
Epoch 6/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2749 - accuracy: 0.9153 - val_loss: 0.2318 - val_accuracy: 0.9290
Epoch 7/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2709 - accuracy: 0.9159 - val_loss: 0.2256 - val_ac

# Arquitectura 6: Mayor correlación individual

In [20]:
X = df_train_dummie[['Type of Travel', 'Class','Flight Distance', 'Inflight wifi service', 'Online boarding','Seat comfort','Inflight entertainment','On-board service', 'Leg room service', 'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.5),
    layers.Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo6.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 8s 3ms/step - loss: 0.5796 - accuracy: 0.8434 - val_loss: 0.3475 - val_accuracy: 0.9034
Epoch 2/100
2590/2590 [==============================] - 6s 3ms/step - loss: 0.3963 - accuracy: 0.8817 - val_loss: 0.3128 - val_accuracy: 0.9106
Epoch 3/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3729 - accuracy: 0.8893 - val_loss: 0.2986 - val_accuracy: 0.9198
Epoch 4/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3656 - accuracy: 0.8914 - val_loss: 0.2865 - val_accuracy: 0.9199
Epoch 5/100
2590/2590 [==============================] - 7s 3ms/step - loss: 0.3552 - accuracy: 0.8938 - val_loss: 0.2780 - val_accuracy: 0.9205
Epoch 6/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3507 - accuracy: 0.8938 - val_loss: 0.2775 - val_accuracy: 0.9185
Epoch 7/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3481 - accuracy: 0.8950 - val_loss: 0.2765 - val_ac

# Arquitectura 7: Nuevos features

In [60]:
X = df_train_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y = df_train_combined['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('Modelo7.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 3s 929us/step - loss: 0.3657 - accuracy: 0.8487 - val_loss: 0.3417 - val_accuracy: 0.8570
Epoch 2/100
2590/2590 [==============================] - 2s 905us/step - loss: 0.3330 - accuracy: 0.8617 - val_loss: 0.3223 - val_accuracy: 0.8664
Epoch 3/100
2590/2590 [==============================] - 2s 888us/step - loss: 0.3211 - accuracy: 0.8664 - val_loss: 0.3235 - val_accuracy: 0.8669
Epoch 4/100
2590/2590 [==============================] - 2s 946us/step - loss: 0.3144 - accuracy: 0.8701 - val_loss: 0.3123 - val_accuracy: 0.8726
Epoch 5/100
2590/2590 [==============================] - 2s 891us/step - loss: 0.3102 - accuracy: 0.8721 - val_loss: 0.3071 - val_accuracy: 0.8724
Epoch 6/100
2590/2590 [==============================] - 2s 834us/step - loss: 0.3070 - accuracy: 0.8730 - val_loss: 0.3030 - val_accuracy: 0.8741
Epoch 7/100
2590/2590 [==============================] - 2s 897us/step - loss: 0.3051 - accuracy: 0.8746 - val_loss: 0

# Arquitectura 8: Nuevos features (+1 capa)

In [62]:
X = df_train_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y = df_train_combined['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo8.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 3s 927us/step - loss: 0.3635 - accuracy: 0.8479 - val_loss: 0.3420 - val_accuracy: 0.8553
Epoch 2/100
2590/2590 [==============================] - 2s 959us/step - loss: 0.3295 - accuracy: 0.8612 - val_loss: 0.3243 - val_accuracy: 0.8644
Epoch 3/100
2590/2590 [==============================] - 2s 846us/step - loss: 0.3188 - accuracy: 0.8660 - val_loss: 0.3149 - val_accuracy: 0.8693
Epoch 4/100
2590/2590 [==============================] - 2s 910us/step - loss: 0.3117 - accuracy: 0.8703 - val_loss: 0.3060 - val_accuracy: 0.8741
Epoch 5/100
2590/2590 [==============================] - 2s 921us/step - loss: 0.3072 - accuracy: 0.8724 - val_loss: 0.3023 - val_accuracy: 0.8732
Epoch 6/100
2590/2590 [==============================] - 2s 897us/step - loss: 0.3041 - accuracy: 0.8737 - val_loss: 0.3098 - val_accuracy: 0.8707
Epoch 7/100
2590/2590 [==============================] - 2s 917us/step - loss: 0.3012 - accuracy: 0.8756 - val_loss: 0

# Arquitectura 9

In [22]:
df_train_dummie.columns

Index(['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [24]:
X = df_train_dummie[['Class','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo9.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2449 - accuracy: 0.8973 - val_loss: 0.1939 - val_accuracy: 0.9196
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1804 - accuracy: 0.9260 - val_loss: 0.1632 - val_accuracy: 0.9346
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1565 - accuracy: 0.9355 - val_loss: 0.1502 - val_accuracy: 0.9392
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1434 - accuracy: 0.9400 - val_loss: 0.1430 - val_accuracy: 0.9421
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1367 - accuracy: 0.9429 - val_loss: 0.1379 - val_accuracy: 0.9441
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1327 - accuracy: 0.9438 - val_loss: 0.1352 - val_accuracy: 0.9426
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1292 - accuracy: 0.9456 - val_loss: 0.1318 - val_ac

In [27]:
X = df_train_dummie[['Class','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo10.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2652 - accuracy: 0.8879 - val_loss: 0.1960 - val_accuracy: 0.9184
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1820 - accuracy: 0.9258 - val_loss: 0.1674 - val_accuracy: 0.9316
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1584 - accuracy: 0.9348 - val_loss: 0.1504 - val_accuracy: 0.9363
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1483 - accuracy: 0.9379 - val_loss: 0.1436 - val_accuracy: 0.9402
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1428 - accuracy: 0.9403 - val_loss: 0.1408 - val_accuracy: 0.9398
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1398 - accuracy: 0.9413 - val_loss: 0.1390 - val_accuracy: 0.9412
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1377 - accuracy: 0.9419 - val_loss: 0.1402 - val_ac

In [29]:
X = df_train_dummie[['Class','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo11.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2311 - accuracy: 0.9050 - val_loss: 0.1773 - val_accuracy: 0.9270
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1653 - accuracy: 0.9319 - val_loss: 0.1510 - val_accuracy: 0.9391
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1466 - accuracy: 0.9391 - val_loss: 0.1383 - val_accuracy: 0.9428
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1375 - accuracy: 0.9433 - val_loss: 0.1329 - val_accuracy: 0.9460
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1310 - accuracy: 0.9452 - val_loss: 0.1274 - val_accuracy: 0.9471
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1258 - accuracy: 0.9476 - val_loss: 0.1266 - val_accuracy: 0.9468
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1224 - accuracy: 0.9485 - val_loss: 0.1271 - val_ac

In [31]:
X = df_train_dummie[['Class','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo12.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2179 - accuracy: 0.9095 - val_loss: 0.1655 - val_accuracy: 0.9333
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1519 - accuracy: 0.9371 - val_loss: 0.1426 - val_accuracy: 0.9414
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1382 - accuracy: 0.9421 - val_loss: 0.1391 - val_accuracy: 0.9406
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1316 - accuracy: 0.9443 - val_loss: 0.1304 - val_accuracy: 0.9432
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1268 - accuracy: 0.9461 - val_loss: 0.1280 - val_accuracy: 0.9448
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1226 - accuracy: 0.9482 - val_loss: 0.1255 - val_accuracy: 0.9480
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1185 - accuracy: 0.9494 - val_loss: 0.1218 - val_ac

In [32]:
X = df_train_dummie[['Class','Type of Travel','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo13.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2129 - accuracy: 0.9140 - val_loss: 0.1621 - val_accuracy: 0.9332
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1481 - accuracy: 0.9400 - val_loss: 0.1412 - val_accuracy: 0.9438
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1337 - accuracy: 0.9451 - val_loss: 0.1279 - val_accuracy: 0.9466
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1256 - accuracy: 0.9485 - val_loss: 0.1231 - val_accuracy: 0.9485
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1205 - accuracy: 0.9498 - val_loss: 0.1209 - val_accuracy: 0.9485
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1168 - accuracy: 0.9518 - val_loss: 0.1201 - val_accuracy: 0.9483
Epoch 7/100
2590/2590 [==============================] - 10s 4ms/step - loss: 0.1145 - accuracy: 0.9525 - val_loss: 0.1164 - val_a

In [34]:
X = df_train_dummie[['Class','Type of Travel','Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness']]
y = df_train_dummie['satisfaction']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[time_callback, early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

model.save('../Modelos/Modelo14.h5')

loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy: {accuracy[-1]*100}')

Epoch 1/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1982 - accuracy: 0.9200 - val_loss: 0.1505 - val_accuracy: 0.9396
Epoch 2/100
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1434 - accuracy: 0.9414 - val_loss: 0.1318 - val_accuracy: 0.9474
Epoch 3/100
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1280 - accuracy: 0.9472 - val_loss: 0.1241 - val_accuracy: 0.9498
Epoch 4/100
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1200 - accuracy: 0.9504 - val_loss: 0.1202 - val_accuracy: 0.9502
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.1141 - accuracy: 0.9528 - val_loss: 0.1190 - val_accuracy: 0.9514
Epoch 6/100
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1105 - accuracy: 0.9543 - val_loss: 0.1171 - val_accuracy: 0.9525
Epoch 7/100
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1070 - accuracy: 0.9561 - val_loss: 0.1146 - val_ac